In [6]:
# Ajout du chemin vers src/
import sys, os
sys.path.append(os.path.abspath(".."))

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from src.signal_model import train_signal_model

# Charger les données enrichies
df = pd.read_csv("../data/processed_orderbook.csv")

# Label : est-ce que le midprice monte dans 5 minutes ?
df["future_midprice"] = df["midprice"].shift(-5)
df["label"] = (df["future_midprice"] > df["midprice"]).astype(int)

# Sélection des features
features = [
    "ofi", "zscore_ofi",
    "ofi_lag1", "ofi_lag2", "ofi_lag3",
    "return_1", "return_5", "volatility_5",
    "midprice"
]
X = df[features].dropna()
y = df["label"].dropna()

# Aligner les index
X, y = X.loc[y.index], y.loc[X.index]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)

# Entraînement du modèle
model = train_signal_model(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Résultats
print(classification_report(y_test, y_pred))

df.to_csv("../data/processed_orderbook_with_label.csv", index=False)


              precision    recall  f1-score   support

           0       0.50      0.62      0.55     32363
           1       0.50      0.38      0.43     32864

    accuracy                           0.50     65227
   macro avg       0.50      0.50      0.49     65227
weighted avg       0.50      0.50      0.49     65227

